In [39]:
import numpy as np
import queue
import copy
import matplotlib.pyplot as plt


In [40]:
# unit: hour
#即核酸检测有两个小时的检测窗口 eg. 9-11 a.m. 
#假设所有人都在这个区间进行检测👉后续可以优化
# simulation(hours)
#arrival rate (/hour) i.e. 2 per min
#service rate (/hour) 这里考虑改进前 即25s左右
#我们假设在正式检测开始之前，已经有人开始排队👉即这里的‘begin', 是一个0-30之间的随机整数
#number of server

begin = np.random.randint(0,30)
c = 1
total_time = 2
IAT_rate = 120
ST_rate = 144*c
rho = IAT_rate/ST_rate
print(begin)

23


In [41]:
# Initialize Parameters
# IAT  empty array for putting random number inter-arrival time
# St   ... for serving time rate
# AT: arrival times
# ST: serve time


qu = queue.Queue()
curr_process = None
IAT = []
ST = []
AT = []
wait_time = []
server_busy = False
list_wait = []
list_delay = []


In [21]:
#队伍长度

L_q = (IAT_rate*IAT_rate)/(ST_rate*(ST_rate-IAT_rate))
print(L_q)

1.3333333333333333


In [5]:
# total queue length in the system
L_s =  L_q + IAT_rate/ST_rate
print(L_s)

2.0


In [22]:
# utilization 
# 当只有一个核酸检测点时 lamda(effective) = lamda
# c bar (expected number of busy serves) = lamda/ miu
c_bar = IAT_rate/ST_rate
utilization = c_bar/c
print(utilization)

0.6666666666666666


In [42]:
# num_processes 即一共有多少个人在检测窗口里来做核酸检测

num_processes = int(np.random.poisson(IAT_rate)* total_time) 
total_people = num_processes + begin
num_processes_served = 0
print(num_processes)
print(begin)
print(total_people)

270
23
293


In [43]:
# Populate Inter-Arrival-Times (IAT)

for i in range(num_processes):
    temp = np.random.exponential(1/IAT_rate)*60*60
    if i==0:
        IAT.append(0)
    else:
        IAT.append(int(temp - temp%1))


In [44]:
IAT_new = []

In [45]:
#Populate new Inter-Arrival-Times (IAT_new)
#即在正式检测开始之前到达的人的inter arrival time=0

for i in range(total_people):
    temp = np.random.exponential(1/IAT_rate)*60*60
    if i<begin:
        IAT_new.append(0)
        i=i+1
    else: 
        IAT_new.append(int(temp-temp%1))

print(IAT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 21, 1, 1, 17, 32, 2, 2, 1, 28, 24, 27, 75, 26, 26, 15, 16, 61, 53, 89, 29, 82, 19, 10, 2, 0, 21, 61, 0, 6, 0, 18, 4, 13, 14, 6, 6, 4, 26, 5, 78, 33, 14, 86, 28, 25, 34, 20, 4, 2, 6, 54, 121, 30, 3, 26, 2, 74, 20, 32, 23, 15, 34, 33, 27, 18, 4, 69, 13, 71, 34, 13, 18, 14, 71, 3, 40, 32, 27, 108, 9, 81, 44, 18, 80, 32, 23, 111, 16, 26, 22, 3, 103, 11, 1, 62, 43, 3, 40, 4, 40, 29, 49, 65, 36, 7, 44, 0, 10, 19, 55, 53, 1, 17, 11, 8, 5, 1, 10, 5, 20, 27, 20, 59, 0, 0, 8, 33, 26, 82, 24, 9, 14, 41, 16, 94, 9, 1, 8, 20, 9, 203, 2, 11, 50, 45, 6, 6, 8, 9, 55, 8, 22, 21, 60, 43, 30, 24, 27, 45, 23, 7, 4, 16, 0, 44, 13, 68, 5, 34, 0, 11, 39, 16, 50, 24, 90, 4, 24, 12, 4, 36, 9, 16, 3, 39, 48, 2, 44, 29, 22, 14, 3, 24, 7, 42, 35, 85, 40, 41, 92, 31, 31, 122, 7, 16, 34, 29, 23, 48, 67, 4, 11, 24, 13, 19, 79, 122, 32, 22, 21, 128, 14, 105, 29, 7, 63, 0, 1, 44, 30, 10, 54, 1, 45, 3, 16, 9, 14, 8, 1, 43, 9, 70, 5, 82, 8, 7, 4, 

In [46]:
# Populate Service-Times (ST) (where ST[i]!=0)

while not len(ST) == total_people:
    temp = np.random.exponential(1/ST_rate)*60*60
    if not int(temp- temp%1)<1:
        ST.append(int(temp - temp%1))

print(ST)

[22, 46, 23, 46, 5, 3, 4, 4, 4, 20, 25, 13, 14, 42, 46, 1, 14, 5, 27, 61, 16, 13, 4, 14, 10, 33, 29, 55, 22, 88, 6, 31, 10, 19, 16, 4, 19, 37, 1, 20, 29, 29, 7, 1, 46, 12, 53, 6, 18, 56, 12, 8, 3, 13, 86, 18, 38, 5, 7, 3, 49, 153, 4, 2, 2, 24, 4, 40, 30, 3, 12, 9, 6, 9, 7, 98, 18, 23, 54, 6, 28, 33, 7, 29, 7, 47, 2, 14, 31, 18, 43, 13, 19, 10, 16, 48, 44, 8, 44, 49, 51, 2, 32, 5, 7, 9, 1, 12, 187, 54, 33, 45, 100, 17, 16, 2, 27, 58, 12, 10, 53, 19, 6, 54, 39, 12, 109, 24, 13, 66, 13, 23, 8, 7, 1, 5, 8, 19, 38, 23, 81, 3, 10, 4, 16, 19, 7, 26, 79, 11, 74, 45, 7, 50, 85, 3, 33, 25, 11, 8, 9, 36, 1, 3, 46, 45, 3, 29, 12, 16, 3, 13, 21, 34, 49, 17, 25, 13, 4, 18, 11, 24, 23, 20, 34, 56, 10, 67, 16, 42, 75, 23, 14, 41, 5, 37, 6, 29, 2, 48, 16, 10, 63, 15, 35, 21, 13, 8, 16, 24, 55, 1, 9, 19, 4, 1, 32, 13, 62, 1, 43, 3, 42, 2, 34, 90, 59, 49, 15, 8, 20, 76, 41, 43, 8, 2, 9, 7, 14, 12, 60, 51, 30, 35, 24, 32, 38, 6, 3, 55, 6, 16, 17, 8, 15, 43, 36, 61, 7, 39, 15, 6, 5, 23, 44, 15, 15, 14, 4, 

In [47]:
# save a copy of ST

ST_copy = copy.deepcopy(ST)

In [48]:
AT_new = []

In [49]:
# Get new Arrival-Times (AT) from IAT starting at t=0 
# 生成到达时间队列
# and initialize Waiting-Times to 0

for i in range(total_people):
    if i<begin:
        AT_new.append(0)
        i=i+1
    else:
        AT_new.append(IAT_new[i]+AT_new[i-1])
    wait_time.append(0)

print(AT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 38, 39, 40, 57, 89, 91, 93, 94, 122, 146, 173, 248, 274, 300, 315, 331, 392, 445, 534, 563, 645, 664, 674, 676, 676, 697, 758, 758, 764, 764, 782, 786, 799, 813, 819, 825, 829, 855, 860, 938, 971, 985, 1071, 1099, 1124, 1158, 1178, 1182, 1184, 1190, 1244, 1365, 1395, 1398, 1424, 1426, 1500, 1520, 1552, 1575, 1590, 1624, 1657, 1684, 1702, 1706, 1775, 1788, 1859, 1893, 1906, 1924, 1938, 2009, 2012, 2052, 2084, 2111, 2219, 2228, 2309, 2353, 2371, 2451, 2483, 2506, 2617, 2633, 2659, 2681, 2684, 2787, 2798, 2799, 2861, 2904, 2907, 2947, 2951, 2991, 3020, 3069, 3134, 3170, 3177, 3221, 3221, 3231, 3250, 3305, 3358, 3359, 3376, 3387, 3395, 3400, 3401, 3411, 3416, 3436, 3463, 3483, 3542, 3542, 3542, 3550, 3583, 3609, 3691, 3715, 3724, 3738, 3779, 3795, 3889, 3898, 3899, 3907, 3927, 3936, 4139, 4141, 4152, 4202, 4247, 4253, 4259, 4267, 4276, 4331, 4339, 4361, 4382, 4442, 4485, 4515, 4539, 4566, 4611, 4634, 4641, 4645, 4661

In [50]:
# simulate M/M/1 queue

for i in range(total_time*60*60):    
    if server_busy:
        for item in list(qu.queue):
            wait_time[item] = wait_time[item] + 1
        ST[curr_process] = ST[curr_process] - 1
        if ST[curr_process] == 0:
            server_busy = False
            num_processes_served = num_processes_served + 1

    for j in range(total_people):
        if i== AT_new[j]:
            qu.put(j)

    if not server_busy and not qu.empty():
        curr_process = qu.get()
        server_busy = True
        


In [51]:
# 总共等待时间
# 总共delay时间 = total time in the system 
# delay = waiting time + serve time
sum_wait = 0
sum_delay = 0

In [52]:
print(num_processes_served)

263


In [53]:
for i in range(num_processes_served):
    sum_wait = sum_wait + wait_time[i]
    sum_delay = sum_delay + wait_time[i] + ST_copy[i]
    
if num_processes_served == 0:
    list_wait.append(0)
    list_delay.append(0)
else:
    list_wait.append(sum_wait/(num_processes_served*60*60))  
    list_delay.append(sum_delay/(num_processes_served*60*60))


In [54]:
print(total_people)
print(begin)
print(num_processes)
print(num_processes_served)

293
23
270
263


In [57]:
#平均等待时间 in hour
print(sum_wait/(num_processes_served*60*60))
print(sum_delay/(num_processes_served*60*60))

# 平均等待时间 in minute
print(sum_wait/(num_processes_served*60))

0.06475390790029573
0.07192015209125475
3.885234474017744


In [58]:
# 单位仍选择为小时
# total cost = service cost + waiting cost 
# ETC(x) = EOC(x) + EWC (x)
# average salary of Beijing: 9227 per month
# 假设每月抛去四个周末，有效工作日为31-8=23天
# 假设每天有效工作时长为8小时
# 根据以上两个假设算时薪
worktime = 23*8
nurse_salary = 6000
salary = 9227
EOC = (nurse_salary/worktime) * total_time*c 
EWC = (salary/worktime)* L_s
ETC = EOC + EWC

print(ETC)


NameError: name 'L_s' is not defined

In [23]:
print((6000/worktime)*2)
print(salary/worktime)

65.21739130434783
50.14673913043478
